Model Workflow
Data Loading &  Processing

Data source: ccxt for real-time and historical data on OHLCV(open, high, low, closing & volume data) for digital currency (e.g. BTC) using a 1d (daily) timeframe 
Feature Engineering : calculate price, volume & volatility-based indicators 
- Price: Simple Moving Average(SMA), EMA, RSI, MACD (for momentum signals)
--> RSI: measure 
- Volume: OBV (On-Balance Volume), Money Flow Index (MFI)
- Volatility: ATR (Average True Range), Bollinger Bands Width(BBW)

Further Feature engineering: implementing Time Series 

- Add weighted averages based on regression of lagged values, using partial autocorrelation (PACF) to identify autoregressive lags for feature selection
- Implement time-series modelling to forecast short-term price movements using ARIMA, & selecting best ARIMA model based on AIC/BIC criteria to obtain parameters (p - autoregressive order & q - simple moving average order)
Filling missing data with interpolation & normalization

Feature Selection through Pre-calculated Technical Indicators 

Trend prediction: inputs - SMA, EMA, using Linear regression (OLS), or support vector machine
Market Regime Detection: inputs - RSI, ATR, SMA using K-means Clustering
Buy/Sell Signals for Binary Classification: inputs - RSI, MACD, OBV, using Random Forest Classifier 

Model Training & Prediction: Machine Learning

Time Series Modelling: ARIMA for short -term price, GARCH for volatility forecasting 
Price Forecasting: LSTM, utilizing keras to implement long-term price forecasting
Strategy Implementation:
Combine signals and implement reinforcement learning to create a trading strategy

Momentum Oscillators(BUY/SELL SIGNALS)
RSI: 
Purpose: Measures the speed & change of price movements.
Use Case: Buy if RSI < 30 (oversold), Sell if RSI > 70 (overbought)
MACD:
Purpose: Identifies trend reversals via moving average crossovers.
Use Case: Buy if MACD crosses above Signal Line, Sell if MACD crosses below Signal Line
(other indicators we have not considered)
Stochastic Oscillator(STOCH)
Purpose: Identifies overbought & oversold conditions based on recent highs/lows.
Use Case: Buy if %K crosses above %D in oversold zone (<20), Sell if %K crosses below %D in overbought zone (>80)
Rate of Change (ROC)
Purpose: Measures the percentage change of price over a given period.
Use Case: Buy when ROC turns positive, Sell when ROC turns negative.

Volitility Indicators 
Bollinger Bands:
Purpose: Measures price volatility relative to a moving average.
Use Case: Buy when price touches lower band, Sell when price touches upper band.
Average True Range (ATR)
Purpose: Measures market volatility by averaging true range values.
Use Case: High ATR → High volatility, Low ATR → Low volatility.

Volume Indicators 
On-Balance Volume (OBV)
Purpose: Cumulative volume indicator that tracks buying/selling pressure.
Use Case: Rising OBV suggests buying pressure, declining OBV suggests selling pressure.
Money Flow Index (MFI)
Purpose: Volume-weighted RSI to assess buying/selling pressure.
Use Case: Buy if MFI < 20, Sell if MFI > 80


In [49]:
#Data Loading
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta

# Initialize exchange
exchange = ccxt.binance()

# Define symbol and timeframe
symbol = 'BTC/USDT'
timeframe = '1d'

# Calculate timestamp for 1 year ago 
end_date = datetime(2025, 2, 28)  # Last day of February 2025
start_date = end_date - timedelta(days=365)  

# Convert to milliseconds (Binance API uses Unix timestamp in milliseconds)
since = int(start_date.timestamp() * 1000)

# Fetch historical OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since)

# Convert to DataFrame
data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data.set_index('timestamp', inplace=True)

# Display the first few rows
print(data.head())
print(data.tail())  # Check the latest date to ensure correct range


                open      high       low     close       volume
timestamp                                                      
2024-02-29  62432.11  63676.35  60364.70  61130.98  78425.07603
2024-03-01  61130.99  63114.23  60777.00  62387.90  47737.93473
2024-03-02  62387.90  62433.19  61561.12  61987.28  25534.73659
2024-03-03  61987.28  63231.88  61320.00  63113.97  28994.90903
2024-03-04  63113.97  68499.00  62300.00  68245.71  84835.16005
                open      high       low     close       volume
timestamp                                                      
2025-02-28  84708.57  85120.00  78258.52  84349.94  83648.03969
2025-03-01  84349.95  86558.00  83824.78  86064.53  25785.05464
2025-03-02  86064.54  95000.00  85050.60  94270.00  54889.09045
2025-03-03  94269.99  94416.46  85117.11  86220.61  59171.10218
2025-03-04  86221.16  86807.98  82641.51  83085.45   6413.62572


In [51]:
#Calculation of Technical Indicators
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator, VolumeWeightedAveragePrice, money_flow_index
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# Calculate technical indicators
def calculate_technical_indicators(data):
    data['SMA'] = SMAIndicator(data['close'], window=14).sma_indicator()
    data['EMA'] = EMAIndicator(data['close'], window=14).ema_indicator()
    data['RSI'] = RSIIndicator(data['close'], window=14).rsi()
    macd = MACD(data['close'], window_fast=12, window_slow=26, window_sign=9)
    data['MACD'] = macd.macd() 
    data['MACD_signal'] = macd.macd_signal()
    data['MACD_hist'] = macd.macd_diff()
    data['OBV'] = OnBalanceVolumeIndicator(data['close'], data['volume']).on_balance_volume()
    data['MFI'] = money_flow_index(data['high'], data['low'], data['close'], data['volume'], window=14)
    data['ATR'] = AverageTrueRange(data['high'], data['low'], data['close'], window=1).average_true_range()
    bands = BollingerBands(data['close'], window=20, window_dev = 2)
    data['upper_band'] = bands.bollinger_hband()
    data['middle_band'] = bands.bollinger_mavg()
    data['lower_band'] = bands.bollinger_lband() 
    data['BBW'] = (data['upper_band'] - data['lower_band']) / data['middle_band']
    return data 
#handle missing values
def handle_missing_data(data):
    data.fillna(method = 'ffill', inplace=True)
    data.fillna(method = 'bfill', inplace = True)
    return data 

# Normalization
def normalize_data(df):
    scaler = MinMaxScaler()
    df[['close', 'SMA', 'EMA', 'RSI', 'MACD', 'OBV', 'ATR']] = scaler.fit_transform(df[['close', 'SMA', 'EMA', 'RSI', 'MACD', 'OBV', 'ATR']])
    return df

btc_data = calculate_technical_indicators(data)
btc_data = handle_missing_data(btc_data)
btc_data = normalize_data(btc_data)

#inspect data
print(btc_data.head()) 
print(btc_data.tail())
print(btc_data.info())


                open      high       low     close       volume      SMA  \
timestamp                                                                  
2024-02-29  62432.11  63676.35  60364.70  0.137369  78425.07603  0.20924   
2024-03-01  61130.99  63114.23  60777.00  0.161456  47737.93473  0.20924   
2024-03-02  62387.90  62433.19  61561.12  0.153779  25534.73659  0.20924   
2024-03-03  61987.28  63231.88  61320.00  0.175371  28994.90903  0.20924   
2024-03-04  63113.97  68499.00  62300.00  0.273716  84835.16005  0.20924   

                 EMA       RSI      MACD  MACD_signal  MACD_hist       OBV  \
timestamp                                                                    
2024-02-29  0.230905  0.889282  0.385692     0.019334  -0.002779  0.644000   
2024-03-01  0.230905  0.889282  0.385692     0.019334  -0.002779  0.693552   
2024-03-02  0.230905  0.889282  0.385692     0.019334  -0.002779  0.667047   
2024-03-03  0.230905  0.889282  0.385692     0.019334  -0.002779  0.697144   

C:\Users\alyss\AppData\Local\Temp\ipykernel_20740\2446309484.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method = 'ffill', inplace=True)
C:\Users\alyss\AppData\Local\Temp\ipykernel_20740\2446309484.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method = 'bfill', inplace = True)


In [ ]:
# Calculate weighted averages using regression on lagged values
def calculate_weighted_averages(data, lags):
    for i in range(1, lags + 1):
        data[f'lag_{i}'] = data['close'].shift(i)
    data.dropna(inplace=True)

    X = data[[f'lag_{i}' for i in range(1, lags + 1)]]
    y = data['close']
    model = LinearRegression()
    model.fit(X, y)
    weights = model.coef_
    data['weighted_avg'] = np.dot(X, weights)

calculate_weighted_averages(btc_data, 5)

# Plot PACF to identify autoregressive lags
plot_pacf(data['close'], lags=20)
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
#Trend prediction